# Notebook error analysis 

## Manual labels of the sampled errors

# Prepare statistics

In [13]:
import importlib
importlib.reload(config)
import config

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [1]:
import pandas as pd
import config
import numpy as np

# what have been labeled
# df_mlerr_labels_1 = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_labeled.xlsx'),
#                                 sheet_name = "Del1",
#                                 keep_default_na=False)
# df_mlerr_labels_1 = df_mlerr_labels_1[df_mlerr_labels_1.label_ML_pipeline != ""]
# df_mlerr_labels_2 = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_labeled.xlsx'),
#                                 sheet_name = "Del2",
#                                 keep_default_na=False)
# df_mlerr_labels_2 = df_mlerr_labels_2[df_mlerr_labels_2.label_ML_pipeline != ""]
# df_mlerr_labels_3 = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_labeled.xlsx'),
#                                 sheet_name = "Del3",
#                                 keep_default_na=False)
# df_mlerr_labels_3 = df_mlerr_labels_3[df_mlerr_labels_3.label_ML_pipeline != ""]
# df_mlerr_labels = pd.concat([df_mlerr_labels_1, df_mlerr_labels_2, df_mlerr_labels_3], ignore_index=True)

df_mlerr_labels = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_labeled.xlsx'),
                                sheet_name = "Del(repaired)-All",
                                keep_default_na=False)

df_mlerr_label_config = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_labeled.xlsx'),
                                sheet_name = "Config",
                                keep_default_na=False)
df_mlerr_label_config_exclude = ["Review_res", "Responsible"]
df_mlerr_label_config.drop(df_mlerr_label_config_exclude, axis=1, inplace=True)

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [2]:
# labels in the config but are not used in manual labeling
for label_key in df_mlerr_label_config:
    print(label_key)
    print([i for i in df_mlerr_label_config[label_key].unique() if i not in df_mlerr_labels[label_key].unique()])
    
    updated_config = [i for i in df_mlerr_label_config[label_key].unique() if i in df_mlerr_labels[label_key].unique()]
    for _ in range(len(df_mlerr_label_config)-len(updated_config)):
        updated_config.append(np.nan)
    for _ in range(len(updated_config)-len(df_mlerr_label_config)):
        df_mlerr_label_config.loc[len(df_mlerr_label_config)] = pd.Series(dtype='float64')
    
    df_mlerr_label_config[label_key] = updated_config

label_root_cause
['misunderstanding of types of objects', 'uninitializated', 'error inside library', '']
label_ML_pipeline
['']
label_if_ML_bug
['']
label_refined_exp_type
['valueerror - row count mismatch']
label_if_runinfo_help
['']
label_if_code_error_align
['']
label_if_error_chain
['unknown', '']
other
['self-raised', '']


In [3]:
# labels used in manual labeling but not in config !!should not happen!!
for label_key in df_mlerr_label_config:
    print(label_key)
    print([i for i in df_mlerr_labels[label_key].unique() if i not in df_mlerr_label_config[label_key].unique()])

label_root_cause
[]
label_ML_pipeline
[]
label_if_ML_bug
[]
label_refined_exp_type
[]
label_if_runinfo_help
[]
label_if_code_error_align
[]
label_if_error_chain
[]
other
[0]


In [4]:
# need to take care of n/a in df_mlerr_label_config (label_refined_exp_type)
# it means the same as the enames

df_mlerr_labels['label_refined_exp_type'] = np.where(df_mlerr_labels['label_refined_exp_type'] == 'n/a', 
                                                     df_mlerr_labels['ename'], 
                                                     df_mlerr_labels['label_refined_exp_type'])
real_refined_exp_types = set(df_mlerr_label_config.label_refined_exp_type).union(set(df_mlerr_labels['label_refined_exp_type'].unique()))
real_refined_exp_types.remove("n/a")
real_refined_exp_types.remove(np.nan)
real_refined_exp_types=list(real_refined_exp_types)
for _ in range(len(df_mlerr_label_config)-len(real_refined_exp_types)):
    real_refined_exp_types.append(np.nan)
for _ in range(len(real_refined_exp_types)-len(df_mlerr_label_config)):
    df_mlerr_label_config.loc[len(df_mlerr_label_config)] = pd.Series(dtype='float64')

df_mlerr_label_config['label_refined_exp_type'] = list(real_refined_exp_types)

In [7]:
assert(df_mlerr_labels.label_refined_exp_type.nunique()==len(df_mlerr_label_config.label_refined_exp_type))
df_mlerr_label_config.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_config.xlsx"), index=False, engine='xlsxwriter')

In [8]:
df_mlerr_labels_g = df_mlerr_labels[df_mlerr_labels["nb_source"]==config.NB_SOURCE["github"]].reset_index(drop=True)
df_mlerr_labels_k = df_mlerr_labels[df_mlerr_labels["nb_source"]==config.NB_SOURCE["kaggle"]].reset_index(drop=True)
assert(len(df_mlerr_labels_g)+len(df_mlerr_labels_k)==len(df_mlerr_labels))

df_mlerr_labels_g.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_g.xlsx"), index=False, engine='xlsxwriter')
df_mlerr_labels_k.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_k.xlsx"), index=False, engine='xlsxwriter')

In [11]:
# plot the distributions, sorted with frequencies
for label_key in df_mlerr_label_config:
    print(df_mlerr_labels[label_key].value_counts().sort_values(ascending=False))
    print("\n")

label_root_cause
misunderstanding of data structure         121
misunderstanding of APIs                   115
nb specific - execution order               91
file/path not found or exist                52
unknown                                     49
wrong implementation                        49
undefined variable                          45
nb specific - previous cell error           44
invalid argument                            42
module not installed                        30
settings(permission, environment)           22
insufficient resources                      19
misunderstanding of ML model                16
typo                                        11
library versions incompatible               10
nb specific - need execute future cells      7
did not import                               5
external control (window closed)             5
undefined function                           4
API change                                   4
change of environment                      

# Map the summarized config

to config and the data

In [1]:
import pandas as pd
import config

# config
df_mlerr_label_config = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_config.xlsx'),
                                keep_default_na=False)
for summed_label_name in config.summed_label_names:
    option_dict = { v: k for k, l in getattr(config, summed_label_name).items() for v in l }
    config_tobe_mapped = df_mlerr_label_config[summed_label_name]
    list_mapped = set(["" if (pd.isnull(item)|(item=="")) else option_dict[item] for item in config_tobe_mapped])
    list_mapped = list(filter(None, list_mapped))
    list_mapped.extend([""]*(len(config_tobe_mapped)-len(list_mapped)))
    df_mlerr_label_config[summed_label_name] = list_mapped
df_mlerr_label_config.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_config_sum.xlsx"), index=False, engine='xlsxwriter')

# data
df_mlerr_labels_g = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_g.xlsx'))
df_mlerr_labels_k = pd.read_excel(config.path_default.joinpath('data_dedup_cluster/df_err_processed_pregroup_sampled4_k.xlsx'))
for summed_label_name in config.summed_label_names:
    option_dict = { v: k for k, l in getattr(config, summed_label_name).items() for v in l }
    df_mlerr_labels_g[summed_label_name] = ["" if (pd.isnull(item)|(item=="")) else option_dict[item] for item in df_mlerr_labels_g[summed_label_name]]
    df_mlerr_labels_k[summed_label_name] = ["" if (pd.isnull(item)|(item=="")) else option_dict[item] for item in df_mlerr_labels_k[summed_label_name]]

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [2]:
df_mlerr_labels_g.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_g_sum.xlsx"), index=False, engine='xlsxwriter')
df_mlerr_labels_k.to_excel(config.path_default.joinpath("data_dedup_cluster/df_err_processed_pregroup_sampled4_k_sum.xlsx"), index=False, engine='xlsxwriter')